In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
import folium
import json
from gurobipy import *

n = 15
customer = [i for i in range(1, n + 1)]
customer2 = [i for i in range(1, n + 1)]
node = [0] + customer
arcos = [(i,j) for i in node for j in node if i != j]
f = open('pelanggan4.json')
data = json.load(f)

# get customer demand and location data
df = pd.DataFrame(data)

# get distance & duration data
distance = np.loadtxt('distance.txt')
duration = np.loadtxt('duration.txt')
vehicle_capacity = 250

# Start time
e = [df['ready_time'][i] for i in range(n + 1)]
e[0] = 0

# Due time
l = [df['due_time'][i] for i in range(n + 1)]
 

# Service time
ser = [df['service_time'][i] for i in range(n + 1)]
ser[0] = 0

M = 100**100  # big number
no_of_vehicles = 3  # free to modify
n = n + 1

In [2]:
m = Model("CVRPTW")
x, y, z = {}, {}, {}  #intialize the decision variable

Restricted license - for non-production use only - expires 2022-01-13


In [3]:
for i in node:
    for j in node: 
        x[i, j] = m.addVar(vtype=GRB.BINARY, name="x%d,%d" % (i, j))
m.update()

'''variable_2: y[j] = cumulative demand covered'''
for j in range(n):
    y[j] = m.addVar(vtype=GRB.INTEGER, name="y%d" % (j))   # cumulative demand satisfied variable
    z[j] = m.addVar(vtype=GRB.INTEGER, name="z%d" % (j))   # cumulative time variable
m.update()
print("------ variable_2 -----")
print(m.display())
print("------ variable_2 -----")

'''constraint_1: sum(x[i,j]) = 1, for i = 1,2,...,32'''  # vehicles leaving each customer node
for i in range(n - 1):
    m.addConstr(quicksum(x[(i + 1, j)] for j in range(n)) == 1)
m.update()

print("------ constraint_1 -----")
print(m.display())
print("------ constraint_1 -----")

''' constraint_2: sum(x[i,j] =1 for j = 1,2,.....,32)'''  # vehicles arriving to each customer node
for j in range(n - 1):
    m.addConstr(quicksum(x[(i, j + 1)] for i in range(n)) == 1)
m.update()

print("------ constraint_2 -----")
print(m.display())
print("------ constraint_2 -----")

'''constraint_3: sum(x[0,j]) = 5'''  # vehicles leaving depot
m.addConstr(quicksum(x[(0, j)] for j in range(n)) == no_of_vehicles)
m.update()

print("------ constraint_3 -----")
print(m.display())
print("------ constraint_3 -----")

'''constraint_4: sum(x[i,0]) = 5'''  # vehicles arriving to depot
m.addConstr(quicksum(x[(i, 0)] for i in range(n)) == no_of_vehicles)
m.update()

print("------ constraint_4 -----")
print(m.display())
print("------ constraint_4 -----")

------ variable_2 -----
Minimize
   <gurobi.LinExpr: 0.0>
Subject To
Binaries
   ['x0,0', 'x0,1', 'x0,2', 'x0,3', 'x0,4', 'x0,5', 'x0,6', 'x0,7', 'x0,8', 'x0,9', 'x0,10', 'x0,11', 'x0,12', 'x0,13', 'x0,14', 'x0,15', 'x1,0', 'x1,1', 'x1,2', 'x1,3', 'x1,4', 'x1,5', 'x1,6', 'x1,7', 'x1,8', 'x1,9', 'x1,10', 'x1,11', 'x1,12', 'x1,13', 'x1,14', 'x1,15', 'x2,0', 'x2,1', 'x2,2', 'x2,3', 'x2,4', 'x2,5', 'x2,6', 'x2,7', 'x2,8', 'x2,9', 'x2,10', 'x2,11', 'x2,12', 'x2,13', 'x2,14', 'x2,15', 'x3,0', 'x3,1', 'x3,2', 'x3,3', 'x3,4', 'x3,5', 'x3,6', 'x3,7', 'x3,8', 'x3,9', 'x3,10', 'x3,11', 'x3,12', 'x3,13', 'x3,14', 'x3,15', 'x4,0', 'x4,1', 'x4,2', 'x4,3', 'x4,4', 'x4,5', 'x4,6', 'x4,7', 'x4,8', 'x4,9', 'x4,10', 'x4,11', 'x4,12', 'x4,13', 'x4,14', 'x4,15', 'x5,0', 'x5,1', 'x5,2', 'x5,3', 'x5,4', 'x5,5', 'x5,6', 'x5,7', 'x5,8', 'x5,9', 'x5,10', 'x5,11', 'x5,12', 'x5,13', 'x5,14', 'x5,15', 'x6,0', 'x6,1', 'x6,2', 'x6,3', 'x6,4', 'x6,5', 'x6,6', 'x6,7', 'x6,8', 'x6,9', 'x6,10', 'x6,11', 'x6,12', 'x6,13'

   R4 : <gurobi.LinExpr: x5,0 + x5,1 + x5,2 + x5,3 + x5,4 + x5,5 + x5,6 + x5,7 + x5,8 + x5,9 + x5,10 + x5,11 + x5,12 + x5,13 + x5,14 + x5,15> = 1.0
   R5 : <gurobi.LinExpr: x6,0 + x6,1 + x6,2 + x6,3 + x6,4 + x6,5 + x6,6 + x6,7 + x6,8 + x6,9 + x6,10 + x6,11 + x6,12 + x6,13 + x6,14 + x6,15> = 1.0
   R6 : <gurobi.LinExpr: x7,0 + x7,1 + x7,2 + x7,3 + x7,4 + x7,5 + x7,6 + x7,7 + x7,8 + x7,9 + x7,10 + x7,11 + x7,12 + x7,13 + x7,14 + x7,15> = 1.0
   R7 : <gurobi.LinExpr: x8,0 + x8,1 + x8,2 + x8,3 + x8,4 + x8,5 + x8,6 + x8,7 + x8,8 + x8,9 + x8,10 + x8,11 + x8,12 + x8,13 + x8,14 + x8,15> = 1.0
   R8 : <gurobi.LinExpr: x9,0 + x9,1 + x9,2 + x9,3 + x9,4 + x9,5 + x9,6 + x9,7 + x9,8 + x9,9 + x9,10 + x9,11 + x9,12 + x9,13 + x9,14 + x9,15> = 1.0
   R9 : <gurobi.LinExpr: x10,0 + x10,1 + x10,2 + x10,3 + x10,4 + x10,5 + x10,6 + x10,7 + x10,8 + x10,9 + x10,10 + x10,11 + x10,12 + x10,13 + x10,14 + x10,15> = 1.0
   R10 : <gurobi.LinExpr: x11,0 + x11,1 + x11,2 + x11,3 + x11,4 + x11,5 + x11,6 + x11,7 + x11,8 

In [4]:
'''constraint_6: time-windows and also eliminating sub-tours'''

for i in range(n - 1):
    # assumption: service starts at 9:00 AM, 9 == 0 minutes, each hour after 9 is 60 minutes plus previous hours
    m.addConstr(z[i + 1] >= e[i + 1])  # service should start after the earliest service start time
    m.addConstr(z[i + 1] <= l[i + 1])  # service can't be started after the latest service start time
    for j in range(n - 1):
        # taking the linear distance from one node to other as travelling time in minutes between those nodes
        m.addConstr(z[i + 1] >= z[j + 1] + (ser[j + 1] + duration[j + 1, i + 1]) * x[j + 1, i + 1] - M*(1-x[j+1, i+1]))
m.update()

print("------ constraint_6 -----")
print(m.display())
print("------ constraint_6 -----")

------ constraint_6 -----
Minimize
   <gurobi.LinExpr: 0.0>
Subject To
   R0 : <gurobi.LinExpr: x1,0 + x1,1 + x1,2 + x1,3 + x1,4 + x1,5 + x1,6 + x1,7 + x1,8 + x1,9 + x1,10 + x1,11 + x1,12 + x1,13 + x1,14 + x1,15> = 1.0
   R1 : <gurobi.LinExpr: x2,0 + x2,1 + x2,2 + x2,3 + x2,4 + x2,5 + x2,6 + x2,7 + x2,8 + x2,9 + x2,10 + x2,11 + x2,12 + x2,13 + x2,14 + x2,15> = 1.0
   R2 : <gurobi.LinExpr: x3,0 + x3,1 + x3,2 + x3,3 + x3,4 + x3,5 + x3,6 + x3,7 + x3,8 + x3,9 + x3,10 + x3,11 + x3,12 + x3,13 + x3,14 + x3,15> = 1.0
   R3 : <gurobi.LinExpr: x4,0 + x4,1 + x4,2 + x4,3 + x4,4 + x4,5 + x4,6 + x4,7 + x4,8 + x4,9 + x4,10 + x4,11 + x4,12 + x4,13 + x4,14 + x4,15> = 1.0
   R4 : <gurobi.LinExpr: x5,0 + x5,1 + x5,2 + x5,3 + x5,4 + x5,5 + x5,6 + x5,7 + x5,8 + x5,9 + x5,10 + x5,11 + x5,12 + x5,13 + x5,14 + x5,15> = 1.0
   R5 : <gurobi.LinExpr: x6,0 + x6,1 + x6,2 + x6,3 + x6,4 + x6,5 + x6,6 + x6,7 + x6,8 + x6,9 + x6,10 + x6,11 + x6,12 + x6,13 + x6,14 + x6,15> = 1.0
   R6 : <gurobi.LinExpr: x7,0 + x7,1 + x7

   R122 : <gurobi.LinExpr: -1e+200 x4,6 + -1.0 z4 + z6> >= -inf
   R123 : <gurobi.LinExpr: -1e+200 x5,6 + -1.0 z5 + z6> >= -inf
   R124 : <gurobi.LinExpr: -1e+200 x6,6> >= -inf
   R125 : <gurobi.LinExpr: -1e+200 x7,6 + z6 + -1.0 z7> >= -inf
   R126 : <gurobi.LinExpr: -1e+200 x8,6 + z6 + -1.0 z8> >= -inf
   R127 : <gurobi.LinExpr: -1e+200 x9,6 + z6 + -1.0 z9> >= -inf
   R128 : <gurobi.LinExpr: -1e+200 x10,6 + z6 + -1.0 z10> >= -inf
   R129 : <gurobi.LinExpr: -1e+200 x11,6 + z6 + -1.0 z11> >= -inf
   R130 : <gurobi.LinExpr: -1e+200 x12,6 + z6 + -1.0 z12> >= -inf
   R131 : <gurobi.LinExpr: -1e+200 x13,6 + z6 + -1.0 z13> >= -inf
   R132 : <gurobi.LinExpr: -1e+200 x14,6 + z6 + -1.0 z14> >= -inf
   R133 : <gurobi.LinExpr: -1e+200 x15,6 + z6 + -1.0 z15> >= -inf
   R134 : <gurobi.LinExpr: z7> >= 219.0
   R135 : <gurobi.LinExpr: z7> <= 1419.0
   R136 : <gurobi.LinExpr: -1e+200 x1,7 + -1.0 z1 + z7> >= -inf
   R137 : <gurobi.LinExpr: -1e+200 x2,7 + -1.0 z2 + z7> >= -inf
   R138 : <gurobi.LinExpr:

In [5]:
''' Either of the constraint_5 or the constrain_6 can eliminate sub-tours independently'''

'''constraint_5: capacity of vehicle and also eliminating sub-tours'''
for j in range(n - 1):
    m.addConstr(y[j + 1] <= vehicle_capacity)
    m.addConstr(y[j + 1] >= df.demand[j + 1])
    for i in range(n - 1):
        m.addConstr(y[j + 1] >= y[i + 1] + df.demand[j + 1] * (x[i + 1, j + 1]) - vehicle_capacity * (1 - (x[i + 1, j + 1])))
m.update()

print("------ constraint_5 -----")
print(m.display())
print("------ constraint_5 -----")

------ constraint_5 -----
Minimize
   <gurobi.LinExpr: 0.0>
Subject To
   R0 : <gurobi.LinExpr: x1,0 + x1,1 + x1,2 + x1,3 + x1,4 + x1,5 + x1,6 + x1,7 + x1,8 + x1,9 + x1,10 + x1,11 + x1,12 + x1,13 + x1,14 + x1,15> = 1.0
   R1 : <gurobi.LinExpr: x2,0 + x2,1 + x2,2 + x2,3 + x2,4 + x2,5 + x2,6 + x2,7 + x2,8 + x2,9 + x2,10 + x2,11 + x2,12 + x2,13 + x2,14 + x2,15> = 1.0
   R2 : <gurobi.LinExpr: x3,0 + x3,1 + x3,2 + x3,3 + x3,4 + x3,5 + x3,6 + x3,7 + x3,8 + x3,9 + x3,10 + x3,11 + x3,12 + x3,13 + x3,14 + x3,15> = 1.0
   R3 : <gurobi.LinExpr: x4,0 + x4,1 + x4,2 + x4,3 + x4,4 + x4,5 + x4,6 + x4,7 + x4,8 + x4,9 + x4,10 + x4,11 + x4,12 + x4,13 + x4,14 + x4,15> = 1.0
   R4 : <gurobi.LinExpr: x5,0 + x5,1 + x5,2 + x5,3 + x5,4 + x5,5 + x5,6 + x5,7 + x5,8 + x5,9 + x5,10 + x5,11 + x5,12 + x5,13 + x5,14 + x5,15> = 1.0
   R5 : <gurobi.LinExpr: x6,0 + x6,1 + x6,2 + x6,3 + x6,4 + x6,5 + x6,6 + x6,7 + x6,8 + x6,9 + x6,10 + x6,11 + x6,12 + x6,13 + x6,14 + x6,15> = 1.0
   R6 : <gurobi.LinExpr: x7,0 + x7,1 + x7

   R119 : <gurobi.LinExpr: -1e+200 x1,6 + -1.0 z1 + z6> >= -inf
   R120 : <gurobi.LinExpr: -1e+200 x2,6 + -1.0 z2 + z6> >= -inf
   R121 : <gurobi.LinExpr: -1e+200 x3,6 + -1.0 z3 + z6> >= -inf
   R122 : <gurobi.LinExpr: -1e+200 x4,6 + -1.0 z4 + z6> >= -inf
   R123 : <gurobi.LinExpr: -1e+200 x5,6 + -1.0 z5 + z6> >= -inf
   R124 : <gurobi.LinExpr: -1e+200 x6,6> >= -inf
   R125 : <gurobi.LinExpr: -1e+200 x7,6 + z6 + -1.0 z7> >= -inf
   R126 : <gurobi.LinExpr: -1e+200 x8,6 + z6 + -1.0 z8> >= -inf
   R127 : <gurobi.LinExpr: -1e+200 x9,6 + z6 + -1.0 z9> >= -inf
   R128 : <gurobi.LinExpr: -1e+200 x10,6 + z6 + -1.0 z10> >= -inf
   R129 : <gurobi.LinExpr: -1e+200 x11,6 + z6 + -1.0 z11> >= -inf
   R130 : <gurobi.LinExpr: -1e+200 x12,6 + z6 + -1.0 z12> >= -inf
   R131 : <gurobi.LinExpr: -1e+200 x13,6 + z6 + -1.0 z13> >= -inf
   R132 : <gurobi.LinExpr: -1e+200 x14,6 + z6 + -1.0 z14> >= -inf
   R133 : <gurobi.LinExpr: -1e+200 x15,6 + z6 + -1.0 z15> >= -inf
   R134 : <gurobi.LinExpr: z7> >= 219.0
   

   R251 : <gurobi.LinExpr: -1e+200 x14,13 + z13 + -1.0 z14> >= -inf
   R252 : <gurobi.LinExpr: -1e+200 x15,13 + z13 + -1.0 z15> >= -inf
   R253 : <gurobi.LinExpr: z14> >= 1018.0
   R254 : <gurobi.LinExpr: z14> <= 2218.0
   R255 : <gurobi.LinExpr: -1e+200 x1,14 + -1.0 z1 + z14> >= -inf
   R256 : <gurobi.LinExpr: -1e+200 x2,14 + -1.0 z2 + z14> >= -inf
   R257 : <gurobi.LinExpr: -1e+200 x3,14 + -1.0 z3 + z14> >= -inf
   R258 : <gurobi.LinExpr: -1e+200 x4,14 + -1.0 z4 + z14> >= -inf
   R259 : <gurobi.LinExpr: -1e+200 x5,14 + -1.0 z5 + z14> >= -inf
   R260 : <gurobi.LinExpr: -1e+200 x6,14 + -1.0 z6 + z14> >= -inf
   R261 : <gurobi.LinExpr: -1e+200 x7,14 + -1.0 z7 + z14> >= -inf
   R262 : <gurobi.LinExpr: -1e+200 x8,14 + -1.0 z8 + z14> >= -inf
   R263 : <gurobi.LinExpr: -1e+200 x9,14 + -1.0 z9 + z14> >= -inf
   R264 : <gurobi.LinExpr: -1e+200 x10,14 + -1.0 z10 + z14> >= -inf
   R265 : <gurobi.LinExpr: -1e+200 x11,14 + -1.0 z11 + z14> >= -inf
   R266 : <gurobi.LinExpr: -1e+200 x12,14 + -1.0 z

   R426 : <gurobi.LinExpr: -306.0 x2,9 + -1.0 y2 + y9> >= -250.0
   R427 : <gurobi.LinExpr: -306.0 x3,9 + -1.0 y3 + y9> >= -250.0
   R428 : <gurobi.LinExpr: -306.0 x4,9 + -1.0 y4 + y9> >= -250.0
   R429 : <gurobi.LinExpr: -306.0 x5,9 + -1.0 y5 + y9> >= -250.0
   R430 : <gurobi.LinExpr: -306.0 x6,9 + -1.0 y6 + y9> >= -250.0
   R431 : <gurobi.LinExpr: -306.0 x7,9 + -1.0 y7 + y9> >= -250.0
   R432 : <gurobi.LinExpr: -306.0 x8,9 + -1.0 y8 + y9> >= -250.0
   R433 : <gurobi.LinExpr: -306.0 x9,9> >= -250.0
   R434 : <gurobi.LinExpr: -306.0 x10,9 + y9 + -1.0 y10> >= -250.0
   R435 : <gurobi.LinExpr: -306.0 x11,9 + y9 + -1.0 y11> >= -250.0
   R436 : <gurobi.LinExpr: -306.0 x12,9 + y9 + -1.0 y12> >= -250.0
   R437 : <gurobi.LinExpr: -306.0 x13,9 + y9 + -1.0 y13> >= -250.0
   R438 : <gurobi.LinExpr: -306.0 x14,9 + y9 + -1.0 y14> >= -250.0
   R439 : <gurobi.LinExpr: -306.0 x15,9 + y9 + -1.0 y15> >= -250.0
   R440 : <gurobi.LinExpr: y10> <= 250.0
   R441 : <gurobi.LinExpr: y10> >= 65.0
   R442 : <g

In [6]:
'''objective function'''
m.setObjective(quicksum(quicksum(x[(i, j)]*distance[(i, j)] for j in range(n)) for i in range(n)),GRB.MINIMIZE)
m.update()

In [7]:
'''optimize'''
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 542 rows, 288 columns and 1862 nonzeros
Model fingerprint: 0x49f96809
Variable types: 0 continuous, 288 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+200]
  Objective range  [2e+01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 300 rows and 32 columns
Presolve time: 0.01s
Presolved: 242 rows, 256 columns, 1112 nonzeros
Variable types: 0 continuous, 256 integer (241 binary)
Found heuristic solution: objective 39428.000000
Found heuristic solution: objective 37380.000000

Root relaxation: objective 1.586835e+04, 59 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

In [8]:
'''retrieve the solution'''
sol_y, sol_x, sol_z = m.getAttr('x', y), m.getAttr('x', x), m.getAttr('x', z)
X, Y, Z = np.empty([n, n]), np.empty([n]), np.empty([n])
for i in range(n):
    Y[i] = sol_y[i]
    Z[i] = sol_z[i]
    for j in range(n):
        X[i, j] = int(sol_x[i, j])
print('\nObjective is:', m.objVal)
print('\nDecision variable X (binary decision of travelling from one node to another):\n', X.astype('int32'))
print('\nDecision variable z:(service start time of every customers in minutes)\n', Z.astype('int32')[1:])
print('\nDecision variable y (cumulative demand collected at every customer node):\n', Y.astype('int32')[1:])



Objective is: 20197.0

Decision variable X (binary decision of travelling from one node to another):
 [[0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]

Decision variable z:(service start time of every customers in minutes)
 [ 987  367 1129  202  325  810  219  860  605  117 1078  140  687 1018
  576]

Decision variable y (cumulative demand collected at every customer node):
 [185 250  82 139  50 204 131 112  56 250 250  33 148 105  66]


In [9]:
route = []
total_dis = []
for h in customer:
    if X[0, h] > 0.9:
        arcos_route = [(0, h)]
        item_dis = distance[0][h]
        j = h
        while j!=0:
            i = j
            for k in node:
                if i!=k and X[i, k] > 0.9:
                    j = k
                    arcos_route.append((i, j))
                    item_dis+= distance[i][j]
                    break
        route.append(arcos_route)
        print(arcos_route)
        print("----------------------")
        total_dis.append(item_dis)


[(0, 9), (9, 3), (3, 8), (8, 4), (4, 1), (1, 2), (2, 0)]
----------------------
[(0, 10), (10, 0)]
----------------------
[(0, 12), (12, 5), (5, 15), (15, 14), (14, 7), (7, 13), (13, 6), (6, 11), (11, 0)]
----------------------


In [10]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    s_time = 0
    for rt in route[n]:
        (i,j) = rt
        s_time += duration[i][j]
        print(str(rt)+" --> "+str(e[j])+" - "+str(l[j])+" --> "+str(s_time))
        s_time += ser[j]
    print("----------")
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))

Rute 0
[(0, 9), (9, 3), (3, 8), (8, 4), (4, 1), (1, 2), (2, 0)] -> 6898.0
(0, 9) --> 605 - 1805 --> 407.0
(9, 3) --> 1129 - 2329 --> 746.0
(3, 8) --> 860 - 2060 --> 1031.0
(8, 4) --> 202 - 1402 --> 1323.0
(4, 1) --> 987 - 2187 --> 1659.0
(1, 2) --> 367 - 1567 --> 1934.0
(2, 0) --> 0 - 5000 --> 2510.0
----------
Rute 1
[(0, 10), (10, 0)] -> 3138.0
(0, 10) --> 117 - 1317 --> 266.0
(10, 0) --> 0 - 5000 --> 846.0
----------
Rute 2
[(0, 12), (12, 5), (5, 15), (15, 14), (14, 7), (7, 13), (13, 6), (6, 11), (11, 0)] -> 10161.0
(0, 12) --> 140 - 1340 --> 377.0
(12, 5) --> 325 - 1525 --> 884.0
(5, 15) --> 576 - 1776 --> 1253.0
(15, 14) --> 1018 - 2218 --> 1536.0
(14, 7) --> 219 - 1419 --> 1959.0
(7, 13) --> 687 - 1887 --> 2201.0
(13, 6) --> 810 - 2010 --> 2563.0
(6, 11) --> 1078 - 2278 --> 3011.0
(11, 0) --> 0 - 5000 --> 3418.0
----------
Total Distace : 20197.0


In [11]:
e

[0,
 987,
 367,
 1129,
 202,
 325,
 810,
 219,
 860,
 605,
 117,
 1078,
 140,
 687,
 1018,
 576]

In [12]:
l

[5000,
 2187,
 1567,
 2329,
 1402,
 1525,
 2010,
 1419,
 2060,
 1805,
 1317,
 2278,
 1340,
 1887,
 2218,
 1776]